In [2]:
import pandas as pd
pd.reset_option('display.float_format')
pd.options.display.float_format = '{:.2f}'.format
from openpyxl import load_workbook

df_movement = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\Movement.xlsx")
df_WC = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\fixed_doc\Warehouse_Risk_Category.xlsx")
df_vessel = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\inventory2.xlsx")
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\inventory.xlsx")
df_vessel = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\inventory2.xlsx")

In [143]:
input= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\unpaid_ref.xlsx")

cols_input = ['Purchase Assignment Reference', 'Purchase Settlement Valuation(USD)', 'Collateral Document Type']
df_input = pd.DataFrame(input, columns=cols_input)


def HC(a):
    return a == "HC"
def transport(a):
    return a != "HC"


df_hc = df_input.loc[df_input['Collateral Document Type'].apply(HC)]
df_transport = df_input.loc[df_input['Collateral Document Type'].apply(transport)]


In [135]:
df_transport

,Purchase Assignment Reference,Purchase Settlement Valuation(USD),Collateral Document Type
1,682698.2.2,3958465.79,BL


In [138]:
#storage

cols_inventory = ['INVENTORY ID','COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'CITY']
cols_wc = ['LOCATION_DESCRIPTION','Country' ,'RISK_CATEGORY']
cols_inventory2 = ['TRANSPORT', 'INVENTORY ID', 'MATERIAL ORIGIN COUNTRIES', 'OPERATIONS OWNER', 'PRODUCT', 'LOCATION', 'PURCHASE COUNTERPARTY']

df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_inventory2_f1 = pd.DataFrame(df_vessel, columns=cols_inventory2)
df_WC_f1 = pd.DataFrame(df_WC, columns=cols_wc)
df_WC_f1['RISK_CATEGORY'] = df_WC_f1['RISK_CATEGORY'].map(lambda x: str(x)[13:])


inventory_combination = pd.merge(df_inventory_f1, df_inventory2_f1, on=['INVENTORY ID'])
final_unpaid_WHS = pd.merge(inventory_combination,df_WC_f1, left_on = 'LOCATION', right_on= 'LOCATION_DESCRIPTION', how='outer')



result_storage = pd.merge(df_hc, final_unpaid_WHS, left_on = 'Purchase Assignment Reference', right_on= 'PURCHASE ASSIGNMENT')

result_storage['Collateral Reference'] = None
result_storage['Purchase Group Company Code'] = 'PTE?'
result_storage[['Purchase Assignment Reference']] = result_storage[['Purchase Assignment Reference']].astype('str')
result_storage['Purchase Trade ID'] = result_storage['Purchase Assignment Reference'].map(lambda x: str(x)[:6])
result_storage.loc[:, 'Purchase Quota Titan ID'] = result_storage['Purchase Assignment Reference'].apply(lambda st: st[st.rfind(".")-8: st.rfind(".")])

result_storage.loc[:, 'Eligible Dry Weight'] = result_storage['DRY WEIGHT']
result_storage.loc[:, 'Eligible Wet Weight'] = result_storage['WET WEIGHT']
result_storage.loc[:, 'Authorized Paid Quantity'] = result_storage['DRY WEIGHT']
result_storage.loc[:, 'Financed Quantity'] = result_storage['DRY WEIGHT']
result_storage.loc[:, 'Inventory Current Value(USD)'] = result_storage['Purchase Settlement Valuation(USD)']

result_storage['Purchase Settlement Valuation(USD)'] = result_storage['Purchase Settlement Valuation(USD)'] * (-1)

result_storage['Purchase Dry Weight UOM'] = 'MT'
result_storage['Purchase Wet Weight UOM'] = 'MT'
result_storage['Delivery In Progress'] = 'N'
result_storage['Receipt In Progress'] = 'N'
result_storage['Financed Ratio'] = '1'

result_storage = result_storage.drop(columns = ['TRANSPORT', 'LOCATION_DESCRIPTION', 'PURCHASE ASSIGNMENT'])

result_storage.columns = ['Purchase Assignment Reference', 'Purchase Settlement Valuation(USD)', 'Collateral Document Type',
       'Purchase Inventory ID', 'Collateral Document Date', 'Purchase Dry Weight', 'Purchase Wet Weight',
       'Warehouse City', 'Purchase Material Origin Country', 'Purchase Traffic Operator', 'Purchase Commodity',
       'Warehouse Reference','Purchase Contractual Counterparty', 'Warehouse Country', 'Warehouse Category', 'Collateral Reference',
       'Purchase Group Company Code', 'Purchase Trade ID',
       'Purchase Quota Titan ID', 'Eligible Dry Weight', 'Eligible Wet Weight',
       'Authorized Paid Quantity', 'Financed Quantity',
       'Inventory Current Value(USD)', 'Purchase Dry Weight UOM',
       'Purchase Wet Weight UOM', 'Delivery In Progress',
       'Receipt In Progress', 'Financing Ratio']

order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', 'Purchase Traffic Operator', 'Purchase Contractual Counterparty',
        'Collateral Document Type', 'Collateral Reference', 'Collateral Document Date', 'Purchase Commodity', 'Purchase Material Origin Country', 'Warehouse Reference', 'Warehouse Category', 'Warehouse Country',
        'Warehouse City', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)', 'Receipt In Progress', 'Delivery In Progress',
        'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorized Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Inventory Current Value(USD)' ]

result_storage = result_storage[order]

In [170]:
#transit
cols_movement = ['ID', 'LOAD DATE','LOAD', 'DESTINATION', 'DESTINATION DATE']
cols_inventory2 = ['TRANSPORT', 'INVENTORY ID', 'MATERIAL ORIGIN COUNTRIES', 'OPERATIONS OWNER', 'PRODUCT', 'PURCHASE COUNTERPARTY', 'MOVEMENT ID', 'MODE', 'RECEIPT INCOTERMS']

df_movement_f1 = pd.DataFrame(df_movement, columns=cols_movement)
df_movement_f1['LOAD'] = df_movement_f1['LOAD'].astype(str)
df_inventory2_f1 = pd.DataFrame(df_vessel, columns=cols_inventory2)

#deliver detail
df_movement_f1.loc[:, 'Load Country'] = df_movement_f1['LOAD'].apply(lambda st: st[st.rfind("(")+1:st.rfind(")")])
df_movement_f1.loc[:, 'Delivery Country'] = df_movement_f1['DESTINATION'].apply(lambda st: st[st.rfind("(")+1:st.find(")")])

inventory_combination = pd.merge(df_inventory_f1, df_inventory2_f1, on=['INVENTORY ID'])
final_unpaid_transit = pd.merge(inventory_combination, df_movement_f1, left_on = 'MOVEMENT ID', right_on = 'ID', how='outer')
result_transport = pd.merge(df_transport, final_unpaid_transit, left_on = 'Purchase Assignment Reference', right_on= 'PURCHASE ASSIGNMENT')

In [171]:

result_transport['Collateral Reference'] = None
result_transport['Purchase Group Company Code'] = 'PTE?'
result_transport[['Purchase Assignment Reference']] = result_transport[['Purchase Assignment Reference']].astype('str')
result_transport['Purchase Trade ID'] = result_transport['Purchase Assignment Reference'].map(lambda x: str(x)[:6])
result_transport.loc[:, 'Purchase Quota Titan ID'] = result_transport['Purchase Assignment Reference'].apply(lambda st: st[st.rfind(".")-8: st.rfind(".")])

result_transport.loc[:, 'Eligible Dry Weight'] = result_transport['DRY WEIGHT']
result_transport.loc[:, 'Eligible Wet Weight'] = result_transport['WET WEIGHT']
result_transport.loc[:, 'BL Date'] = result_transport['COLLATERAL DOCUMENT DATE']
result_transport.loc[:, 'Authorized Paid Quantity'] = result_transport['DRY WEIGHT']
result_transport.loc[:, 'Financed Quantity'] = result_transport['DRY WEIGHT']
result_transport.loc[:, 'Inventory Current Value(USD)'] = result_transport['Purchase Settlement Valuation(USD)']

result_transport['Purchase Settlement Valuation(USD)'] = result_transport['Purchase Settlement Valuation(USD)'] * (-1)

result_transport['Purchase Dry Weight UOM'] = 'MT'
result_transport['Purchase Wet Weight UOM'] = 'MT'
result_transport['Delivery In Progress'] = 'N'
result_transport['Receipt In Progress'] = 'N'
result_transport['Financed Ratio'] = '1'

In [166]:
result_transport.columns

Index(['Purchase Assignment Reference', 'Purchase Settlement Valuation(USD)',
       'Collateral Document Type', 'INVENTORY ID', 'COLLATERAL DOCUMENT DATE',
       'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'CITY', 'TRANSPORT',
       'MATERIAL ORIGIN COUNTRIES', 'OPERATIONS OWNER', 'PRODUCT',
       'PURCHASE COUNTERPARTY', 'MOVEMENT ID', 'MODE', 'RECEIPT INCOTERMS',
       'ID', 'LOAD DATE', 'LOAD', 'DESTINATION', 'DESTINATION DATE',
       'Load Country', 'Delivery Country', 'Collateral Reference',
       'Purchase Group Company Code', 'Purchase Trade ID',
       'Purchase Quota Titan ID', 'Eligible Dry Weight', 'Eligible Wet Weight',
       'BL Date', 'Authorized Paid Quantity', 'Financed Quantity',
       'Inventory Current Value(USD)', 'Purchase Dry Weight UOM',
       'Purchase Wet Weight UOM', 'Delivery In Progress',
       'Receipt In Progress', 'Financed Ratio'],
      dtype='object')

In [172]:
result_transport = result_transport.drop(columns = ['PURCHASE ASSIGNMENT', 'ID', 'MOVEMENT ID', 'CITY'])

In [177]:
result_transport.columns = ['Purchase Assignment Reference', 'Purchase Settlement Valuation(USD)',
       'Collateral Document Type', 'Purchase Inventory ID', 'Collateral Document Date',
       'Purchase Dry Weight', 'Purchase Wet Weight', 'Vessel Name',
       'Purchase Material Origin Country', 'Purchase Traffic Operator', 'Purchase Commodity',
       'Purchase Contractual Counterparty', 'Purchase Transport Mode', 'Purchase Incoterm', 'Load  Date',
       'Load Location', 'Delivery Location', 'Expected Delivery Date', 'Load Country',
       'Delivery Country', 'Collateral Reference',
       'Purchase Group Company Code', 'Purchase Trade ID',
       'Purchase Quota Titan ID', 'Eligible Dry Weight', 'Eligible Wet Weight','BL Date',
       'Authorized Paid Quantity', 'Financed Quantity',
       'Inventory Current Value(USD)', 'Purchase Dry Weight UOM',
       'Purchase Wet Weight UOM', 'Delivery In Progress',
       'Receipt In Progress', 'Financing Ratio']

In [178]:
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', 'Purchase Traffic Operator', 'Purchase Contractual Counterparty',
        'Collateral Document Type', 'Collateral Reference', 'Collateral Document Date', 'Purchase Commodity', 'Purchase Material Origin Country', 'Purchase Incoterm', 'Purchase Transport Mode', 'Vessel Name', 'Load  Date',
        'Load Location', 'Load Country','BL Date', 'Expected Delivery Date', 'Delivery Location', 'Delivery Country' ,'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)', 'Receipt In Progress', 'Delivery In Progress',
        'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorized Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Inventory Current Value(USD)' ]

In [179]:
result_transport = result_transport[order]